In [1]:
# Importing required libraries
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Load the dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Define the number of features to select
k = 5

In [6]:
# Feature Selection using SelectKBest
selector_k_best = SelectKBest(score_func=f_regression, k=k)
X_train_k_best = selector_k_best.fit_transform(X_train_scaled, y_train)
X_test_k_best = selector_k_best.transform(X_test_scaled)

In [7]:
# Feature Selection using RFE
selector_rfe = RFE(estimator=LinearRegression(), n_features_to_select=k)
X_train_rfe = selector_rfe.fit_transform(X_train_scaled, y_train)
X_test_rfe = selector_rfe.transform(X_test_scaled)

In [8]:
# Dimensionality Reduction using PCA
pca = PCA(n_components=k)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [9]:
# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Random Forest": RandomForestRegressor()
}


In [10]:
# Function to train and evaluate a model
def train_evaluate_model(model, X_train, X_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

In [11]:
# Initialize a dictionary to store results
results = {model_name: {"SelectKBest": None, "RFE": None, "PCA": None} for model_name in models}

In [12]:
# Train each model using each feature selection method and store results
for model_name, model in models.items():
    results[model_name]["SelectKBest"] = train_evaluate_model(model, X_train_k_best, X_test_k_best)
    results[model_name]["RFE"] = train_evaluate_model(model, X_train_rfe, X_test_rfe)
    results[model_name]["PCA"] = train_evaluate_model(model, X_train_pca, X_test_pca)

In [13]:
# Display the results in the specified table format
print("| Model               | MSE with SelectKBest | MSE with RFE | MSE with PCA |")
print("|---------------------|----------------------|--------------|--------------|")
for model_name, scores in results.items():
    print(f"| {model_name:<19} | {scores['SelectKBest']:<20} | {scores['RFE']:<12} | {scores['PCA']} |")

| Model               | MSE with SelectKBest | MSE with RFE | MSE with PCA |
|---------------------|----------------------|--------------|--------------|
| Linear Regression   | 0.6382565441555919   | 0.5667695170781499 | 0.7431025684653442 |
| Ridge Regression    | 0.6382133029769735   | 0.5667326028214025 | 0.7430998408273355 |
| Lasso Regression    | 1.3106960720039365   | 1.3106960720039365 | 1.3106960720039365 |
| Random Forest       | 0.4487779633474012   | 0.2386777326507045 | 0.5551448307277324 |
